# Data Science en pratique

Arthur Llau: arthur@flowlity.com

# Cours 6 : Algorithmes et techniques avancées

### Objectif du cours:
-  Présentation de quelques algorithmes et implémentations avancées.
-  Présenter rapidemment l'importance des variables
-  TP de mise en pratique (as usual)

Tout d'abord : 

!pip install lightgbm, xgboost, castboost, eli5, lofo-importance

## 1 - Algorithmes et implémentations avancées

In [ ]:
#Sans wrapper cela ressemble à cela.
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression

train, target = make_regression(n_features=10,n_samples=10000)
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.2)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)


### 1.1 - TabNet 

Original paper (Arik et al., 2020) : https://arxiv.org/pdf/1908.07442.pdf

TabNet est un modèle d'apprentissage automatique pour les données tabulaires, intégrant le principe d'attention (comme dans les transformers !) attention pour se concentrer sur des caractéristiques clés du dataset.

Il combine des aspects des arbres de décision et des réseaux de neurones, offrant à la fois interprétabilité et capacité à capturer des relations complexes.



![](TabNet.png)

In [ ]:
from pytorch_tabnet.tab_model import  TabNetRegressor

model = TabNetRegressor(verbose=0,seed=42)
model.fit(X_train=x_train, y_train=y_train.reshape(-1,1),
              eval_set=[(x_valid, y_valid.reshape(-1,1))],
              patience=50, max_epochs=200,
              eval_metric=['rmse'])

#Prediction
preds = model.predict(x_test)
print ('TabNet scoring : {}'.format(mean_squared_error(y_test,preds, squared=False)))

### 1.2  Boosting & XtremBoosting

Les trois algorithmes décrits ci-dessous sont des méthodes de gradient boosting. LightGBM et Xgboost sont des implémentations dites d'extrême boosting, c'est-à-dire qu'elles sont construites de manière à utiliser du mieu possible les ressources computationnelles et d'un point de vu théorique la différentielle est d'ordre 2.

Il est important de noté que ces méthodes sont implémentés de manière propre mais possède également un wrapper scikit-learn. 


Un autre atout majeur de ces implémentations est l'utilisation de jeux de validation pour obtenir le nombre d'itérations optimal. A chaque itération on regarde si les performances sur le jeu de validation sont améliorés, sinon on arrête. On prend alors la dernière itération la plus performante. Evidemment, il est possible de choisir un nombre d'itération pour lequel il n'y a d'améliorations. (voir Tp)

#### 1.2.1 XGBoost

Original paper (T.Chen, 2016) : https://arxiv.org/abs/1603.02754

Il existe plusieurs diffèrences notable avec l'implémentation du GBT classique : 
-  Ne tiens pas compte des valeurs manquantes...
- Le modèle est mieux régularisé, on contrôle mieux l'overfitting, ce qui explique ces bonnes performances
- On trouve des paramètres de pénalisation $L^1$ et $L^2$, qui n'existe pas dans la version original
- D'autres methodes que les arbres CART peuvent être utilisés: des régréssions linéaire (Linear Boosting), ou des arbres DART (Dropouts meet Multiple Additive Regression Trees, http://proceedings.mlr.press/v38/korlakaivinayak15.pdf)
- On peut résoudre la grande majorité des problématique industrielles  avec  cette implémentation, de la régréssion au ranking, en passant par de la classification multi-classe.
-  Très customisable

On retrouve les même paramètres que pour l'implémentation du GBT dans sklearn. Attention, le nom n'est pas forcément le même, je vous invite à lire la doc.
Cependant, il en existe quelques autres qui sont importants:
- **booster** : type de méthode à booster 
- **objective** : l'objectif du modèle (régéssion, etc.)
- **tree_method** : méthodes de construction des arbres (exact, approx, histogramme)
- **eval_metric** : choix de la métrique d'évaluation

Pour utiliser la version sklearn : "from xgboost import XGBRegressor" (ou XGBClassifier).

/!\ Tous les paramètres ne sont pas disponibles dans le wrapper sklearn.

In [ ]:
##### Sans wrapper cela ressemble à cela.
import xgboost as xgb

#Definition des objets d'apprentissage et test
dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_valid, y_valid)
dtest = xgb.DMatrix(x_test, y_test)

#Apprentissage
param = {'boost':'linear',
         'learnin_rate':0.1,
         'max_depth': 5, 
         'objective': 'reg:linear',
          'eval_metric':'rmse'}
num_round = 500
bst = xgb.train(param, dtrain,  num_boost_round = num_round, evals=[(dvalid, 'validation')], verbose_eval=100)


#Prediction
preds = bst.predict(dtest)
print ('Xgboost scoring : {}'.format(mean_squared_error(y_test,preds, squared=False)))

#### 1.2.2 Lightgbm

Original paper (Microsoft Research, 2017) : https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree

De même on trouve des différences majeurs avec les anciennes implémentations:
-   Construction des arbres verticales et non horizontale, i.e, l'algorithme choisi la feuille avec la meilleure loss pour grandir.
-  Très efficace et rapide sur les données sparses, et les gros volumes de données.
-  Comme pour xgboost, d'autres booster sont disponible comme les random forest.
-  Il consomme très peu de ressources mémoire.
-  Résolution de n'importe quel type de problématique.
-  Très customisable

On retrouve les même paramètres que pour l'implémentation du GBT dans sklearn. Attention, le nom n'est pas forcément le même, je vous invite à lire la doc.
Cependant, il en existe quelques autres qui sont importants - les noms varient par rapport à xgb:
- **boosting_type** : type de méthode à booster 
- **task** : l'objectif du modèle (régéssion, etc.)
- **device** : CPU/GPU
- **metric** : choix de la métrique d'évaluation

Pour utiliser la version sklearn : "from lightgbm import LGBMRegressor" (ou LGBMClassifier).

/!\ Tous les paramètres ne sont pas disponibles dans le wrapper sklearn.

In [ ]:
import lightgbm as lgb

lgb_train = lgb.Dataset(x_train, y_train)
lgb_valid = lgb.Dataset(x_valid, y_valid)
lgb_test = lgb.Dataset(x_test, y_test, reference=lgb_train)

params = {
   
    'objective': 'regression',
    'metric': {'l2', 'rmse'},

}

print('Start training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=lgb_valid,
                verbose_eval=100,
                early_stopping_rounds=20) # Arret si 5 iterations sans gain de performance


print('Start predicting...')
preds = gbm.predict(x_test, num_iteration=gbm.best_iteration)
print('LGBM scoring :', mean_squared_error(y_test, preds, squared=False))

#### 1.2.3 CatBoost

Original paper (Yandex, 2017) : https://arxiv.org/abs/1706.09516

Dernier né de la firme russe Yandex, CatBoost est une implémentation similaire à celle de xgboost et de lightgbm mais qui a la particularité de tenir compte des variables catégorielles pour l'apprentissage. L'algorithme va construire à partir des variables catégorielles diverses statistiques* et, tenir compte de celles-ci pour l'apprentissage. On trouve quelque différences notables avec les deux autres algorithmes présentés ci-dessous:
- Très performant mais très lent.
- Pas d'autre booster que les arbres de disponibles
- Deux taches : régréssion ou classification
- Plusieurs paramètres pour l'apprentissage des variables catégorielles.
- Paramètres de détéction de l'overfitting
- Il y a une interface graphique super sexy...

*Voir https://tech.yandex.com/catboost/doc/dg/concepts/algorithm-main-stages_cat-to-numberic-docpage/

Quelques paramètres importants :
- **iterations** : nombre d'arbres
- **eval_metric** : la métrique d'évaluation
- **ctr** : paramètre de transformation des variables catégorielles
- **fold_permutation_block_size** : nombre de permutations des catégorielles

Pour utiliser la version sklearn : "from catboost import CatBoostRegressor" (ou CatBoostClassifier).
Et pour utiliser l'apprentissage sur les variables catégorielles, ajouter dans *fit* cat_features = [index des features catégorielles].

/!\ Tous les paramètres ne sont pas disponibles dans le wrapper sklearn.

In [ ]:
import catboost as cat

train_pool = cat.Pool(x_train, y_train)
valid_pool = cat.Pool(x_valid, y_valid)
test_pool = cat.Pool(x_test)

param = {'logging_level':'Silent'}
model = cat.CatBoost(param)
model.fit(train_pool,eval_set=valid_pool) 

preds = model.predict(test_pool)
print('Cat scoring :', mean_squared_error(y_test, preds, squared=False))

Notez que les trois implémentations ci-dessus sont également portable sur GPU.

Chaque algorithme aura des performances différentes selon le jeu de données et le problème à traiter, à vous de choisir (et de tester) le meilleur modèle.

### 1.3 - Regularized Greedy Forest

Original paper (R. Johnson et al., 2014) : https://arxiv.org/pdf/1109.0887.pdf

Popularisé il y a peu les RGF sont une sorte de mélange entre des forêts aléatoire et du boosting. Résumé de manière naïve, c'est une fôret aléatoire où chaque arbre est régularisé, et l'ensemble de la fôret également.

C'est un algorithme puissant mais complexe, je vous invite si cela vous intéresse à lire le papier original.

La liste des paramètres est disponible ici : https://github.com/fukatani/rgf_python

Deux modèles sont implémentés : RGFRegressor et RGFClassifier du package rgf_python.

### 1.4 Autres algorithmes plus ou moins célèbres

Vous pouvez également vous penchez sur les algorithmes suivant : 
- Rotation Forest : http://ieeexplore.ieee.org/document/1677518/ PCA sur des K-Folds pour chaque arbre de la forêt
- Adaptative Hybrid Extreme Rotation Forest (AdaHERF) : https://www.ncbi.nlm.nih.gov/pubmed/24480062 Rotation Forest + Extreme Learning
- FTRL (Follow-the-Regularized-Leader) : https://static.googleusercontent.com/media/research.google.com/fr//pubs/archive/37013.pdf


**Aparté sur les Neural Net**

En pratique sur certains jeux de données ils sont équivalent en terme de performance aux meilleurs méthodes de boosting. 

Cependant plusieurs contraintes :
- la recherche d'architecture
- la calibration 
- la reproductibilité
- la mise en production

## Et le non supervisé ?

Il existe également d'autres algorithme que ceux vu en cours pour la réduction de dimension et le clustering :
- T-SNE & variantes (Réduction de dimensions, https://lvdmaaten.github.io/tsne/) : Minimiser la divergence de KL
- TruncatedSVD (Réduction de dimensions, voir cours de C.Boyer)
- SparsePCA (Réduction de dimensions, https://arxiv.org/abs/1211.1309) : PCA sur des données sparses
- HDDBScan (Clustering, https://link.springer.com/chapter/10.1007%2F978-3-642-37456-2_14) : Clustering par densité
- AutoEncoder for Clustering (https://arxiv.org/abs/2102.07472)

Pour découvrir de nouveaux algorithmes je vous invite à consulter les divers forums de machine learning indiqué au premier cours, ainsi que les papiers des conférences NIPS, ICML & MLConf...

## 2 - Feature importance & Feature Selection

### 2.1 - Feature importance

#### 2.1.0 Définition

On parle de feature importance presque uniquement dans le cas des algorithmes de machine learning basé sur les arbres CART.
L'importance d'une variable, n'est autre que le poids - selon une mesure donnée - qu'apporte cette variable dans l'apprentissage. 
Pour les arbres CART on retrouve la définition suivante :


*Les arbres aléatoires sont une règle de décision interprétable pour la prédiction des classes.En effet, les variables de découpe sont choisies selon leur capacité à classifier les données. De ce fait, une variable permettant de découper les premiers nœuds a un pouvoir discriminant plus important qu’une variable apparaissant dans les dernières découpes (ou n’apparaissant pas dans l’arbre). * B.Gregorutti- https://tel.archives-ouvertes.fr/tel-01146830/document

Dans les méthodes d'ensemble, l'importance des features est calculé généralement de la manière suivante:
On compte le nombre de fois où la variable est séléctionné pour séparer un noeud, pondéré par l'amélioration du modèle à chaque séparation. Puis on moyenne les résultats.

Pour résumer : variable très utilisée qui sépare bien les données = variable importante.

#### 2.1.1 Permutation importance

La permutation importance est une autre méthode pour calculer l'importance des features, elle a les particularités suivantes:
- Calcul rapide
- Beaucoup utilisée et comprise
- Consistante


__Comment ça marche ?__

- 0. Découper en train test
- 1. Faire apprendre un modèle sur nos données
- 2. Mesurer les performances du modèle
- 3. Mélanger les valeurs d'une colonne d'un jeu de test puis prédire
- 4. Evaluer de nouveau les performances

Plus la métrique souffre de ce changement plus la variable est importante.

-> Remettre en place la colonne, puis répéter sur toutes les variables du jeu de données

#### 2.1.2 LOFO

LOFO (Leave One Feature Out) Importance calcule l'importance pour un ensemble de variable, basé sur une métrique choisi, avec un modèle choisi. Itérativement une variable va être enlevé de l'ensemble puis on va réévaluer les performances du modèles avec un schéma de validation choisi.

https://github.com/aerdem4/lofo-importance

Avantages :

- Se généralise bien aux données non observées
- Indépendant du modèle
- Donne une importance négative aux variables qui dégrade les performances en les incluants.


### 2.2 - Feature Selection

Il est parfois nécéssaire d'effectuer une séléction des variables utilisées dans le modèle pour plusieurs raisons:
- Trop de corrélations entre certaines variables
- Trop de variables ce qui entraine un temps de calcul excessif
- Expertise métier
- Problème de dimension
- Variable à la distribution étrange
- Et enfin, importance faible

Il existe alors plusieurs technique pour séléctionner les variables, autre que les critères d'Akkaike etc.


Voici la liste de ceux que j'utilise en pratique:

- VarianceThreshold : On supprime les features avec une variance dépassant un certain seuil.
- SelectKBest : On ne séléctionne que les $k$ features les plus important
- RFECV : On supprime récursivement des features et on regarde les performance du modèle

Tous ces modèles de séléction sont disponible dans sklearn.


## TP de mise en pratique


Les données proviennent du Kaggle suivant : https://www.kaggle.com/c/house-prices-advanced-regression-techniques

La variable à prédire est la "SalePrice".


In [ ]:
import pickle
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns,time

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


warnings.filterwarnings('ignore')

**1.1** Le fichier datasets.pkl contient différents dataframe, ouvre le fichier puis observer les clés.

In [ ]:
datasets = pd.read_pickle("...")

In [ ]:
datasets.keys() # 3 dataframe et les deux targets

**2.1** Extrayez les targets

In [ ]:
# Récupérer les targets
y_train = ...
y_test = ...

**2.2** Extrayez le train et le test restant pour le jeu de données "full_clean"


In [ ]:
train_full = ...
test_full = ...

**3** Quelle mesure de performance devrions nous utiliser ? Implémentez la.

La rmse semble pertinente, implémentons-la !

In [ ]:
def rmse(preds, targets):
    ...

**4** Comparez xgboost, lightgbm, catboost, extratrees, et une régression au choix sur le jeu de données full adéquat à chaque algorithme, sans chercher à optimiser les paramètres des modèles. Donnez également le temps de calcul nécessaire à l'apprentissage. 
Faites une fonction !

In [ ]:

def training_and_evaluation(regressor_list, x_train, y_train, x_test, y_test):
    ...

In [ ]:
training_and_evaluation(regressors_list, train_full, y_train, test_full, y_test)


**6.1** Choisissez deux de vos modèles, faites leur apprendre à nouveau le jeu de données et affichez l'importance des 20 features les plus importants, qu'observez vous ?

**6.2** Utiliser lofo pour afficher l'importance des features grâce  à la methode leave one out. Regardez la doc de lofo !

In [ ]:
from lofo import LOFOImportance, Dataset, plot_importance
from sklearn.model_selection import KFold

cv = ...

dataset = ...

lofo_imp = ...
importances = ...
plot_importance(importances, figsize=(12, 20))


**7** Entrainez à nouveau les modèles mais seulement sur les 20 features les plus importants d'un des deux modèles. Donnez également le temps de calcul nécessaire à l'apprentissage et les performances sur le jeu de test. Qu'observez-vous ?

In [ ]:
#Recuper les 20 features les plus important de catboost
best_features = ...
train_importance = train_full[best_features]
test_importance = test_full[best_features]

In [ ]:
training_and_evaluation(regressors_list, train_importance,y_train,test_importance,y_test)

Ne prendre qu'un certain nombre des features importante dégrade les performances de nos modèles, mais cependant le temps de calcul est grandement amélioré. Il faut donc trouver un compromis entre performance et temps de calcul, selon le problème donné.

**8** Jouez avec les paramètres des modèles, et déterminer le plus performant sur le jeu de test.

In [ ]:
regressors_list = ...
training_and_evaluation(regressors_list,train_full,y_train,test_full,y_test)


**9** Utilisez un ensemble de validation pour connaitre le nombre d'itérations optimal de boosting avec Xgboost puis avec catboost avec ou sans wrapper sklearn.

In [ ]:
...

In [ ]:
...

**10** Comment améliorer les résultats ?